In [1]:
import numpy as np
import pandas as pd
import re
import random
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
def limpar_texto(texto):
    texto = texto.lower()  # Converte tudo para minúsculas
    texto = re.sub(r'[^\w\s]', ' ', texto)  # Remove pontuações
    texto = re.sub(r'\d+', ' ', texto)      # Remove números
    texto = re.sub(r'\s+', ' ', texto)      # Substitui múltiplos espaços por um só
    return texto.strip()

In [6]:
texto_demo = """
A modelagem de linguagens é uma área da linguística computacional e da inteligência artificial que tem como objetivo representar, 
processar e gerar textos em linguagem humana de forma automática. Modelos de linguagem são algoritmos que aprendem padrões estatísticos 
a partir de grandes corpora (coleções de textos) para prever a próxima palavra de uma sentença ou avaliar a probabilidade de uma sequência de palavras.
Existem diferentes tipos de modelos de linguagem, sendo os **n-gramas** os mais simples e tradicionais. Um modelo de n-grama utiliza a frequência com que
sequências de *n* palavras ocorrem em um corpus para estimar probabilidades. Por exemplo, um **modelo unigram** considera apenas a frequência individual
das palavras; um **bigram** leva em conta pares de palavras consecutivas; e um **trigram** analisa trios de palavras.
Apesar da simplicidade, esses modelos são bastante eficazes em tarefas como correção ortográfica, sugestão de palavras, tradução automática e sistemas de 
reconhecimento de fala. No entanto, apresentam limitações quanto à captura de dependências de longo alcance, o que motivou o desenvolvimento de modelos mais 
avançados baseados em redes neurais, como os modelos RNN, LSTM, e os atuais **transformers**, utilizados em sistemas como o ChatGPT.
A modelagem de linguagens é fundamental para a criação de tecnologias mais acessíveis, inteligentes e capazes de interagir de forma natural com os seres humanos.
"""

In [4]:
texto_tratado = limpar_texto(texto_demo)
print(texto_tratado)

a modelagem de linguagem natural é uma técnica importante no processamento de linguagem natural os modelos de linguagem podem ser usados para prever a próxima palavra em uma sequência existem diversos tipos de modelos de linguagem incluindo modelos baseados em n grams e modelos neurais os modelos baseados em n grams são mais simples mas ainda muito úteis em diversas aplicações


In [7]:
palavras = limpar_texto(texto_demo).split()
print("Quantidade de palavras:", len(palavras))
print("Exemplo:", palavras[:50])

NameError: name 're' is not defined

In [6]:
class ModeloNGram:
    def __init__(self, n):
        self.n = n
        self.contagens = defaultdict(Counter)
        self.modelo = {}

    def treinar(self, palavras):
        for i in range(len(palavras) - self.n + 1):
            contexto = tuple(palavras[i:i+self.n-1])
            alvo = palavras[i+self.n-1]
            self.contagens[contexto][alvo] += 1

        for contexto, alvos in self.contagens.items():
            total = sum(alvos.values())
            self.modelo[contexto] = {
                palavra: freq / total for palavra, freq in alvos.items()
            }

    def proxima_palavra(self, contexto):
        contexto = tuple(contexto[-(self.n-1):]) if self.n > 1 else ()
        if contexto in self.modelo:
            palavras = list(self.modelo[contexto].keys())
            probs = list(self.modelo[contexto].values())
            return np.random.choice(palavras, p=probs)
        return None

    def gerar_texto(self, contexto_inicial, quantidade=15):
        if isinstance(contexto_inicial, str):
            contexto_inicial = contexto_inicial.split()
        if len(contexto_inicial) < self.n - 1:
            raise ValueError(f"Contexto inicial deve ter pelo menos {self.n - 1} palavras")
        texto = list(contexto_inicial)
        for _ in range(quantidade):
            prox = self.proxima_palavra(texto)
            if not prox:
                break
            texto.append(prox)
        return ' '.join(texto)

In [22]:
modelo_1gram = ModeloNGram(1)
modelo_2gram = ModeloNGram(2)
modelo_3gram = ModeloNGram(3)
modelo_7gram = ModeloNGram(7)

modelo_1gram.treinar(palavras)
modelo_2gram.treinar(palavras)
modelo_3gram.treinar(palavras)
modelo_7gram.treinar(palavras)

In [4]:
def calcular_perplexidade(modelo, texto_teste):
    tokens = limpar_texto(texto_teste).split()
    log_prob = 0
    total = 0
    for i in range(modelo.n - 1, len(tokens)):
        contexto = tuple(tokens[i - modelo.n + 1:i]) if modelo.n > 1 else ()
        alvo = tokens[i]
        prob = modelo.modelo.get(contexto, {}).get(alvo, 1e-10)
        log_prob += np.log2(prob)
        total += 1
    return 2 ** (-log_prob / total) if total > 0 else float('inf')

texto_teste = """
A modelagem de linguagens é uma área da linguística computacional e da inteligência artificial que tem como objetivo representar, 
processar e gerar textos em linguagem humana de forma automática. Modelos de linguagem são algoritmos que aprendem padrões estatísticos 
a partir de grandes corpora (coleções de textos) para prever a próxima palavra de uma sentença ou avaliar a probabilidade de uma sequência de palavras.
Existem diferentes tipos de modelos de linguagem, sendo os **n-gramas** os mais simples e tradicionais. Um modelo de n-grama utiliza a frequência com que
sequências de *n* palavras ocorrem em um corpus para estimar probabilidades. Por exemplo, um **modelo unigram** considera apenas a frequência individual
das palavras; um **bigram** leva em conta pares de palavras consecutivas; e um **trigram** analisa trios de palavras.
Apesar da simplicidade, esses modelos são bastante eficazes em tarefas como correção ortográfica, sugestão de palavras, tradução automática e sistemas de 
reconhecimento de fala. No entanto, apresentam limitações quanto à captura de dependências de longo alcance, o que motivou o desenvolvimento de modelos mais 
avançados baseados em redes neurais, como os modelos RNN, LSTM, e os atuais **transformers**, utilizados em sistemas como o ChatGPT.
A modelagem de linguagens é fundamental para a criação de tecnologias mais acessíveis, inteligentes e capazes de interagir de forma natural com os seres humanos.
"""

print("Perplexidade:")
print("Unigram:", calcular_perplexidade(modelo_1gram, texto_teste))
print("Bigram:", calcular_perplexidade(modelo_2gram, texto_teste))
print("Trigram:", calcular_perplexidade(modelo_3gram, texto_teste))
print("setagram:", calcular_perplexidade(modelo_7gram, texto_teste))

Perplexidade:


NameError: name 'modelo_1gram' is not defined

In [9]:
def mostrar_distribuicao(modelo, contexto=None):
    if modelo.n == 1:
        contexto = ()
    if contexto is None or contexto not in modelo.modelo:
        contexto = list(modelo.modelo.keys())[0]
    palavras = list(modelo.modelo[contexto].keys())
    probs = list(modelo.modelo[contexto].values())
    plt.figure(figsize=(10,5))
    plt.bar(palavras[:10], probs[:10])
    titulo = "Distribuição unigram" if modelo.n == 1 else f"Contexto: {' '.join(contexto)}"
    plt.title(titulo)
    plt.ylabel("Probabilidade")
    plt.xticks(rotation=45)
    plt.show()

mostrar_distribuicao(modelo_1gram)
mostrar_distribuicao(modelo_2gram)
mostrar_distribuicao(modelo_7gram)

NameError: name 'modelo_1gram' is not defined

In [10]:
def distancia_edicao(s1, s2):
    if len(s1) < len(s2):
        return distancia_edicao(s2, s1)
    if len(s2) == 0:
        return len(s1)
    anterior = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        atual = [i + 1]
        for j, c2 in enumerate(s2):
            custos = [
                anterior[j + 1] + 1,
                atual[j] + 1,
                anterior[j] + (c1 != c2)
            ]
            atual.append(min(custos))
        anterior = atual
    return anterior[-1]

def sugerir_palavra(palavra, modelo):
    if palavra in modelo.modelo[()]:
        return palavra
    candidatos = []
    for voc in modelo.modelo[()]:
        dist = distancia_edicao(palavra, voc)
        if dist <= 2:
            prob = modelo.modelo[()].get(voc, 0)
            candidatos.append((voc, dist, prob))
    if not candidatos:
        return palavra
    candidatos.sort(key=lambda x: (x[1], -x[2]))
    return candidatos[0][0]

In [25]:
palavras_erradas = ["modelgem", "linguaem", "natual", "processment"]
print("Correções sugeridas:")
for erro in palavras_erradas:
    print(f"{erro} → {sugerir_palavra(erro, modelo_1gram)}")

Correções sugeridas:
modelgem → modelgem
linguaem → linguagem
natual → natual
processment → processment


In [26]:
print("Textos gerados:")
print("Unigram:", modelo_1gram.gerar_texto([], 30))
print("Bigram:", modelo_2gram.gerar_texto(["existem"], 15))
print("Trigram:", modelo_3gram.gerar_texto(["diferentes", "tipos"], 50))
print("setagram:", modelo_7gram.gerar_texto(["de", "modelos",'de', 'linguagem', 'sendo', 'os', 'n', 'gramas'], 100))

Textos gerados:
Unigram: um tipos de trios trios a de grama modelo existem que bigram que linguagem n modelo diferentes tradicionais trigram um n de estimar apenas a mais apenas tradicionais estimar ocorrem
Bigram: existem diferentes tipos de palavras um modelo de palavras ocorrem em conta pares de n grama
Trigram: diferentes tipos de modelos de linguagem sendo os n gramas os mais simples e tradicionais um modelo de n palavras ocorrem em um corpus para estimar probabilidades por exemplo um modelo unigram considera apenas a frequência individual das palavras um bigram leva em conta pares de palavras consecutivas e um trigram analisa
setagram: de modelos de linguagem sendo os n gramas os mais simples e tradicionais um modelo de n grama utiliza a frequência com que sequências de n palavras ocorrem em um corpus para estimar probabilidades por exemplo um modelo unigram considera apenas a frequência individual das palavras um bigram leva em conta pares de palavras consecutivas e um trigram a